# Day 4 - Self-Paced Practice: Test-Driven Debugging

**Objective:** Reinforce the concepts of Day 4 by using an LLM to perform a complete test-driven debugging workflow on a new, isolated problem.

**Estimated Time:** 45 minutes

**Introduction:**
A powerful workflow for fixing bugs is to first write a test that fails because of the bug, and then fix the code to make the test pass. This is a core principle of Test-Driven Development (TDD). In this lab, you will use an AI co-pilot to execute this entire workflow on a provided piece of buggy code.

## 1. Setup

Let's start by setting up our environment and initializing the LLM client.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, clean_llm_output

# Models with strong reasoning and code understanding are best for this task.
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

## 2. The Buggy Code

Here is a Python function that is supposed to calculate the total cost of a shopping cart. It takes a list of items (as dictionaries) and an optional discount percentage. However, it contains a subtle logical bug.

In [ ]:
buggy_code = """ 
def calculate_total(items, discount_percent=0):
    \"\"\"Calculates the total cost of a shopping cart with an optional discount.\"\"\" 
    total = sum(item['price'] * item['quantity'] for item in items)
    if discount_percent > 0:
        # The bug is here: This calculates the discount amount, not the final price.
        total = total * (discount_percent / 100)
    return total
"""

### ⭐ Deeper Dive: The Test-Driven Development (TDD) Cycle

The workflow in this lab is a direct application of the **Red-Green-Refactor** cycle from Test-Driven Development, a professional software development practice:

1.  **Red:** Write a test that fails. This is our AI-generated `failing_test_code`. The test fails because the bug exists, proving that the bug is real and reproducible.
2.  **Green:** Write the minimum amount of code required to make the failing test pass. This is the `fixed_code` our AI will generate.
3.  **Refactor:** With the test now passing, you can safely clean up or improve the code without changing its behavior, confident that the test will catch any new bugs.

Using AI to accelerate the Red and Green steps can make this powerful practice much faster and more accessible.

## 3. Your Task

Use an LLM to perform a three-step debugging process.

1.  **Analyze the Bug:** Prompt the LLM to explain the logical error in the `buggy_code`.
2.  **Write a Failing Test:** Prompt the LLM to write a `pytest` function that will fail because of the bug.
3.  **Fix the Code:** Prompt the LLM to fix the `buggy_code`, using the failing test as context for what the correct behavior should be.

#### 💡 Pro-Tip: Fixing Bugs with Tests as Context

The most effective way to ask an LLM to fix a bug is to provide it with **both the buggy code and a failing test case.**

Why? The failing test gives the LLM an unambiguous, executable definition of what the *correct* output should be. Instead of just saying "the discount is wrong," you're saying, `"assert calculate_total(cart, 10) == 90.0"`. This concrete example removes ambiguity and dramatically increases the chances that the LLM will produce the correct fix on the first try.

In [ ]:
# --- Task 1: Analyze and explain the bug --- 
analyze_bug_prompt = f"""
You are a senior Python developer acting as a code reviewer.
Analyze the following Python function and explain the logical error in it. Be concise.

```python
{buggy_code}
```
"""
print("--- Analyzing Bug ---")
bug_explanation = get_completion(analyze_bug_prompt, client, model_name, api_provider)
print(bug_explanation)

# --- Task 2: Write a failing pytest test ---
failing_test_prompt = f"""
You are a QA engineer writing tests with pytest.
Based on the following Python function, write a test function named `test_calculate_total_with_discount` that will FAIL because of the bug in the discount calculation. The test should assert the correct expected value.

```python
{buggy_code}
```
"""
print("\n--- Generating Failing Test ---")
failing_test_code_raw = get_completion(failing_test_prompt, client, model_name, api_provider)
failing_test_code = clean_llm_output(failing_test_code_raw, language='python')
print(failing_test_code)

# --- Task 3: Fix the code based on the failing test ---
fix_code_prompt = f"""
You are a senior Python developer tasked with fixing a bug.

The following function is buggy:
<buggy_code>
```python
{buggy_code}
```
</buggy_code>

The following pytest test fails when run against it, demonstrating the bug:
<failing_test>
```python
{failing_test_code}
```
</failing_test>

Your task is to fix the `calculate_total` function so that the provided test will pass. Output only the corrected function.
"""
print("\n--- Generating Fixed Code ---")
fixed_code_raw = get_completion(fix_code_prompt, client, model_name, api_provider)
fixed_code = clean_llm_output(fixed_code_raw, language='python')
print(fixed_code)

# Save the final corrected code
if fixed_code:
    save_artifact(fixed_code, "app/day4_sp_fixed_shopping_cart.py")

## Lab Conclusion

Excellent! You have successfully used an AI co-pilot to execute a professional, test-driven debugging workflow. This process—identifying a bug, writing a test to prove it exists, and then fixing the code to make the test pass—is a highly effective way to systematically improve code quality and prevent regressions. This is a skill you can apply to any codebase.